In [3]:
from os import listdir
import os
import numpy as np
import operator

In [4]:
def MNNB(sub_dict, words, total):
    prob = -1
    for word in words:
        if(word in sub_dict):
            prob += (np.log2(sub_dict[word] + 1) - np.log2(sub_dict["total_count"] + total))
    prob += (np.log2(sub_dict["total_count"]) - np.log2(total))
    return prob

In [5]:
def find_prediction(dictionary, file_path):
    l_words = read_file(file_path)
    words = clean_data(l_words)
    prob = -1
    best_prob = -1000000
    news_grp = ''
    dict_keys = dictionary.keys()
    for key in dict_keys:
        if(key != 'total'):
            prob = MNNB(dictionary[key], words, dictionary["total"])
            print(prob)
            if(prob > best_prob):
                news_grp = key
                best_prob = prob
    return news_grp

In [6]:
def predict(dictionary, path):
    files_path = os.listdir(path)
    predictions = []
    for file_path in files_path:
        pred = find_prediction(dictionary, path + "\\" + file_path)
        predictions.append(pred)
    return predictions

In [7]:
def files_path(base_path):
    folders_name = listdir(base_path)
    folders_path = []
    for folder_name in folders_name:
        folders_path.append(base_path + "\\" + folder_name)
    file_path = []
    for folder_path in folders_path:
        l = os.listdir(folder_path)
        for i in l:
            file_path.append(folder_path + "\\" + i)
    return folders_name, folders_path, file_path

In [8]:
def clean_data(l):
    stopwords = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the", " ", ".", ",", ":", ";", "-", "--", "?", "/", "\\", '']
    for word in list(l):
        if(word in stopwords or not word.isalpha()):
            l.remove(word)
    return l

In [9]:
def read_file(path):
    l = []
    i = 0
    myfile = open(path, 'r')
    for line in myfile.readlines():
        if (i > 12):
            x = (line.strip().split(" "))
            for w in x:
                l.append(w.lower())
        i += 1
    return l

In [10]:
def build_word_dictionary(list_words):
    dictionary_words = {}
    for word in list_words:
        if(word not in dictionary_words.keys()):
            dictionary_words[word] = 1
        else:
            dictionary_words[word] += 1
    return dictionary_words

In [11]:
def make_final_dict(dict_imp_words, folders_name, folders_path):
    dictionary = {}
    wd_keys = dict_imp_words.keys()
    for i in range(len(folders_name)):
        feature = folders_name[i]
        dictionary["total"] = len(dict_imp_words)
        dictionary[feature] = dict_imp_words.copy()
        path = folders_path[i]
        folder_i_files = os.listdir(path)
        for folder_i_file in folder_i_files:
            file_words = read_file(path + "\\" + folder_i_file)
            pure_words = clean_data(file_words)
            for word in pure_words:
                if(word not in wd_keys):
                    continue
                else:
                    dictionary[feature][word] += 1
        dictionary[feature]["total_count"] = sum(dictionary[feature].values())
    return dictionary

In [12]:
def dict_build(base_path, n):
    folders_name, folders_path, file_path = files_path(base_path)
    list_words = []
    for file in file_path:
        file_words = read_file(file)
        pure_words = clean_data(file_words)
        list_words += pure_words
    dictionary_words = build_word_dictionary(list_words)
    sorted_dict = sorted(dictionary_words.items(), key=operator.itemgetter(1), reverse = True)
    imp_words = sorted_dict[0:n]
    dict_imp_words = {}
    for i in range(len(imp_words)):
        if(imp_words[i][0] not in dict_imp_words.keys()):
            x = imp_words[i][0]
            dict_imp_words[x] = 0
    dictionary = make_final_dict(dict_imp_words, folders_name, folders_path)
    return dictionary

In [13]:
dictionary = dict_build("C:\\Users\\jitender\\Downloads\\20_newsgroups_train", 5500)

In [14]:
dictionary

{'total': 5500,
 'alt.atheism': {'i': 3959,
  'just': 593,
  'like': 514,
  'people': 800,
  'know': 484,
  'think': 673,
  'article': 463,
  'does': 587,
  'use': 181,
  'new': 169,
  'good': 311,
  'make': 351,
  'x': 35,
  'time': 284,
  'want': 168,
  'did': 230,
  'say': 538,
  'way': 308,
  'used': 161,
  'need': 142,
  'really': 226,
  'believe': 420,
  'right': 157,
  'going': 120,
  'said': 316,
  'god': 614,
  'using': 88,
  'work': 96,
  'better': 207,
  'government': 37,
  'got': 90,
  'sure': 124,
  'things': 230,
  'come': 153,
  'read': 240,
  'number': 93,
  'problem': 110,
  'look': 141,
  'information': 69,
  'point': 223,
  'little': 145,
  'university': 75,
  'probably': 109,
  'different': 189,
  'file': 18,
  'years': 80,
  'available': 17,
  'set': 103,
  'data': 14,
  'long': 124,
  'try': 163,
  'tell': 120,
  'help': 53,
  'power': 67,
  'best': 88,
  'lot': 81,
  'thing': 142,
  'program': 16,
  'windows': 0,
  'question': 151,
  'state': 70,
  'run': 18,
  '

In [16]:
p = "C:\\Users\\jitender\\Downloads\\mini_newsgroups\\alt.atheism"
pr = predict(dictionary, p)

-341.0949983895802
-362.13283478916264
-348.8656479601489
-355.61230093942635
-350.048385212537
-369.2324360268561
-358.7357409232173
-355.2443193833847
-344.51325700438633
-359.36631319323004
-361.82002839546584
-333.7731631919797
-354.7275803770145
-342.43367860422785
-342.9142831212789
-338.02742662833793
-349.36476768000904
-330.54071517520583
-324.323439784536
-335.4291299649362
-103.66455790308113
-118.16932471276257
-118.28437454426624
-115.64137685226538
-113.40248783040379
-114.07530235043868
-117.2776100828442
-119.91134214331555
-117.113828360169
-121.4192426856097
-120.45337600078904
-117.1668370154687
-116.67501338306369
-114.89696979757484
-114.56847280581621
-110.71724925063718
-115.04578464069661
-117.63014089441404
-114.07211069164374
-113.63302280625898
-247.19104864145794
-285.41098988389746
-280.4924537956012
-289.6298974251399
-290.44931653753747
-281.64542365515223
-293.2285472587189
-281.8737319144667
-278.40854134408085
-270.997774327246
-279.284794457753
-281.9

-513.5116397566587
-509.45245417247764
-521.0956573325708
-475.992504263378
-302.18121350249305
-359.99380930859866
-352.19995015914424
-355.6392619896204
-344.0101365534904
-360.24741450139317
-358.03748839249056
-345.48716048927747
-335.92853562255846
-343.3939977586178
-351.55495340846494
-351.11673856360636
-350.3812498074559
-332.5616907473841
-349.6080188864492
-317.9016849020349
-340.7289685610578
-342.5767384080627
-339.8921282286193
-316.0802043236151
-515.6408936883291
-580.5371484872176
-587.2863714856699
-589.4105454376789
-577.9550034338426
-597.5486111666262
-572.1862229105684
-562.7650873352491
-560.4960343290212
-552.2060728471506
-573.8270306528336
-555.3140289374863
-579.3417978410451
-559.8266204354036
-559.8172025716076
-537.5379229094619
-540.577861868395
-554.5957367156012
-551.3528432337415
-528.4896705481209
-298.1853795611496
-321.5848010799699
-320.5835500065179
-316.4147370130065
-313.8141522306358
-327.6538760127591
-316.70898397163324
-314.01477410790716
-3

-577.2185379153864
-695.2762676025383
-690.5686570754303
-682.4299227556844
-679.0036023820193
-715.1258713406281
-691.7410946546532
-662.1088626844253
-664.8321343863984
-654.1143970465738
-664.7366672719825
-694.4786195158194
-680.9029528621066
-667.6669459526362
-687.408290757074
-543.1901957045379
-652.3397712591775
-646.286108179302
-644.3040106348158
-577.8359360290538
-798.2440382760506
-913.5619710071998
-899.2692755004929
-914.9173202677451
-908.9800812624062
-937.9417636303485
-917.1092226584399
-874.4992666717421
-895.6377971718983
-907.9804442053236
-929.312059322907
-894.0417800669114
-889.9743093619842
-894.3521836362694
-866.4987792165946
-861.0872092847887
-886.4601707547766
-896.5986768985356
-870.2620044063153
-794.4646752453198
-234.5137631546439
-282.28890086025825
-272.0321957535439
-278.54553033548973
-275.24216084732905
-286.2527909351254
-275.0787167344539
-264.97216732453035
-268.21818772010346
-270.0381250438067
-272.25959676553794
-276.1818127125245
-266.1473

-588.0833608635654
-676.3832828187363
-657.0433305461992
-659.2118162446833
-655.4638247789468
-658.7245229657548
-686.2302721590781
-659.787667566143
-651.4407402018813
-655.3924081675777
-663.3281530952751
-647.983732494831
-653.6335438419604
-649.1726861312675
-664.4225764506781
-591.8708276686289
-623.6865938006734
-624.5672485250849
-618.446610434319
-582.682588388486


In [47]:
# find how many files have been predicted right out of 200
total = 0
# all the different groups of news
news_g = ["alt.atheism","comp.graphics","comp.os.ms-windows.misc","comp.sys.ibm.pc.hardware","comp.sys.mac.hardware","comp.windows.x","misc.forsale","rec.autos","rec.motorcycles","rec.sport.baseball","rec.sport.hockey","sci.crypt","sci.electronics","sci.med","sci.space","soc.religion.christian","talk.politics.guns","talk.politics.mideast","talk.politics.misc","talk.religion.misc"]
# number of files predicted correctly out of 200 from each group
for i in range(len(news_g)):
    for p in pr:
        if(p == news_g[i]):
            total += 1
    print(t)

115
117
118
119
119
119
119
122
124
125
125
125
126
131
133
150
154
160
161
200
